## Общая характеристика API
Аббревиатура расшифровывается как Application Programming Interface, или интерфейс для программирования приложений.
В случае веб-приложений, API может отдавать данные в отличном от стандартного HTML формате, благодаря чему им удобно пользоваться при написании собственных приложений. Сторонние общедоступные API чаще всего отдают данные в одном из двух форматов: XML или JSON.

На основе API строятся такие вещи, как карты 2GIS, всевозможные мобильные и десктопные клиенты для Twitter и Vkontakte. Все их функции стали возможными именно благодаря тому, что соответствующие сервисы имеют качественные и детально документированные API.

API VK описан в документации https://vk.com/dev и более конкретно https://vk.com/dev/api_requests.

Можно попробовать сделать запрос из браузера вручную https://api.vk.com/method/getProfiles?uid=66748. В качестве ответа на этот запрос мы получим ответ в формате json:

`{"response":[{"uid":66748,"first_name":"Oleg","last_name":"Illarionov","hidden":1}]}`



> P.S.:

> Почитайте про формат строки [URL](https://ru.wikipedia.org/wiki/URL).

>> Параметры – строка запроса с передаваемыми на сервер (методом GET) параметрами. Начинается с символа ?, разделитель параметров — знак &. Пример: ?параметр_1=значение_1&параметр_2=значение_2&параметр3=значение_3

> Почитайте про формат [JSON](https://ru.wikipedia.org/wiki/JSON).

## Создание приложения

Для того что бы использовать все возможности VK API нужно получить access token аккаунта. Для этого Вам потребуется [создать Standalone-приложение](https://vk.com/editapp?act=create).

После того как мы создали приложение Вы можете найти его разделе [Приложения](https://vk.com/apps?act=manage).

## Приватный токен

Многие методы VK API предполагают наличие приватного токена, который необходимо передать в качетсве параметра при выполнении запроса. Процесс получения токена описан в документации: https://vk.com/dev/access_token

> Осторожно! Токен называется приватным не просто так. Обладающий им человек может производить самые разные действия от Вашего имени. Не говорите его никому.

Если вкратце, то Вам поднадобится id Вашего приложения и [список прав доступа](https://vk.com/dev/permissions), которые Вы хотите предоставить пользователю API. Затем необходимо указать эти данные в качестве параметров в URL следующего формата
https://oauth.vk.com/authorize?client_id={APP_ID}&scope={APP_PERMISSIONS}&response_type=token, подтвердить своё намерение предоставить доступ и скопировать токент из поля URL открывшегося окна.

Покажем на примере:

In [98]:
APP_ID = 6217902
PERMISSIONS = ""
AUTH_URL = f"https://oauth.vk.com/authorize?client_id={APP_ID}&scope={PERMISSIONS}&response_type=token"
AUTH_URL

'https://oauth.vk.com/authorize?client_id=6217902&scope=&response_type=token'

Пройдя по этой ссылки мы попадём настраницу с адресом https://oauth.vk.com/blank.html#access_token=5614afdcc2bcd42cea3d9c5edc130101dd4be5339b484131870dc12337e5b74b94411de69f0996379dd6b&expires_in=86400&user_id=18779389, где `5614afdcc2bcd42cea3d9c5edc130101dd4be5339b484131870dc12337e5b74b94411de69f0996379dd6b` — искомый access token. Сохраним его.

In [101]:
TOKEN = "5614afdcc2bcd42cea3d9c5edc130101dd4be5339b484131870dc12337e5b74b94411de69f0996379dd6b"

## Выполнение запросов к VK API

После получения приватного токена можно смело выполнять запросы к API, используя нужные Вам [методы](https://vk.com/dev/methods). Формат запроса следующий: https://api.vk.com/method/METHOD_NAME?PARAMETERS&access_token=ACCESS_TOKEN. Например, для получения информации о пользователе c id 777 нужно выполнить следующй запрос:

In [105]:
res = requests.get(
    "https://api.vk.com/method/users.get",
    params={"user_ids": 777,
            "fields": "nickname, screen_name, sex, bdate, city, country, timezone, counters, photo_medium",
            "access_token": TOKEN}).json()
res

{'response': [{'bdate': '23.10',
   'city': 22168,
   'counters': {'albums': 8,
    'audios': 360,
    'followers': 5702,
    'friends': 5242,
    'gifts': 125,
    'groups': 7,
    'mutual_friends': 0,
    'notes': 8,
    'online_friends': 382,
    'pages': 0,
    'photos': 146,
    'subscriptions': 119,
    'user_photos': 3,
    'videos': 34},
   'country': 139,
   'first_name': 'Teodor',
   'last_name': 'Master',
   'nickname': '†',
   'photo_medium': 'https://pp.userapi.com/c606428/v606428777/58d0/8seTeA35knk.jpg',
   'screen_name': 'id777',
   'sex': 2,
   'uid': 777}]}

## Ограничения VK API
**Ограничение количества запросов через vk api — не более трёх запросов в секунду.**

>There can be maximum 3 requests to API methods per second from a client. 

>Maximum amount of server requests depends on the app's users amount. 
If an app has less than 10 000 users, 5 requests per second, up to 100 000 – 8 requests, up to 1 000 000 – 20 requests, 1 000 000+ – 35 requests. 

>If one of this limits is exceeded, the server will return following error: 'Too many requests per second'. 

>If your app's logic implies many requests in a row, check the execute method. 

>Except the frequency limits there are quantitative limits on calling the methods of the same type. By obvious reasons we don't provide the exact limits info. 

>On excess of a quantitative limit access to a particular method will require captcha (see captcha_error). After that it may be temporarily limited (in this case the server doesn't answer on particular method's requests but easily processes any other requests).

Сделать паузу при выполнении какой-либо операции в Python можно с помощью функции `slepp` из модуля `time`, которой необходимо передать количество секунд, на которые присотановится выполнение программы:

In [106]:
import time

for i in range(5):
    time.sleep(.5)
    print(i)

0
1
2
3
4


## Ошибки при работе с VK API

Достаточно часто VK API возвращает [ошибки](https://vk.com/dev/errors), которые необходимо отлавливать. В ином случае к концу сбора данных Вы можете обнаружить, что с таким трудом собранная база данных абсолютно непригода к работе, поскольку вместо информации о респонденте Вы имеет следующие данные:

Также существует множество всяких другх тонкостей при работе с VK API. Например, для получения списка друзей пользователя необходимо использовать метод [`friends.get`](https://vk.com/dev/friends.get), который может возвращать как просто список друзей, так и детальную информацию о каждом друге, в зависимости от того, указан ли параметр `fields` (если не указан, возвращает просто список id). Причем если параметр `fields` указан, то за один запрос Вы не можете получить информацию о более, чем 5000 человек.

## Самостоятельная работа

Постройте эго-сеть друзей пользователя с id 777 в формате Adjacency lists. Найдите в этой сети пользователя с найбольшей степенью центральности (Degree centrality), исключая, конечно, самого пользователя с id 777.

> 
![](https://courses.cs.washington.edu/courses/csep521/99sp/lectures/lecture01/img029.JPG)